### Webscrape, analyze, and map PFAS and spatial data from DOD's PFAS website
#### One table has the location data, the other has the PFAS sample data

In [1]:
import requests
import os
import json
from requests import get
import pickle
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px

os.chdir(r'C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\python\PACT Act - MET\final version January 2024')
cwd = os.getcwd()
print("Current working directory is:", cwd)

Current working directory is: C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\python\PACT Act - MET\final version January 2024


In [ ]:
#making request using request package and viewing json
#pulling in the metadata which as latitude and longitude for each military base, which is needed for future mapping
meta_url = 'https://providers.acq.osd.mil/pfasapi/odata/installation?'
metadata = requests.get(meta_url)
metadata = metadata.json()

In [ ]:
#converting from json to pandas dataframe
metadata_df = pd.DataFrame.from_dict(metadata['value'])

#there is at least one duplicate, Letterkenney, so removing duplicates
metadata_df = metadata_df.sort_values('InstallationName', ascending=True).drop_duplicates('InstallationName')

metadata_df.head()

#optional storing metadata for later use
#%store metadata_df

In [ ]:
#pull in raw sample data
#website: 'https://providers.acq.osd.mil/pfasapi/odata/installation?$select=Id,DodComponent,DodComponentId,InstallationName,
#                  InstallationType,State&expand=FinalData'
sample_url = 'https://providers.acq.osd.mil/pfasapi/odata/installation?$select=Id,DodComponent,DodComponentId,InstallationName,InstallationType,State&expand=FinalData'

sample_data = requests.get(sample_url)
sample_data_json = sample_data.json()
sample_data_df = pd.DataFrame.from_records(sample_data_json['value'])
sample_data_df.head()

In [ ]:
#accessing sublist called "Final Data"
sublist_final = sample_data_df.loc[:,['FinalData']]#.head()
sublist_final.head()

In [ ]:
for i in sublist_final.columns:
    mod_final=sublist_final.explode(i)
mod_final.head()

In [ ]:
#convert final list to pandas dataframe
flat_dct=pd.json_normalize(json.loads(mod_final.to_json(orient="records")))

#pd.set_option('display.max_rows', 5000)
flat_dct.head()

In [ ]:
#drop rows that are empty and FinalData column which is empty
final_data_nona = flat_dct[flat_dct['FinalData.InstallationName'].notna()]
final_data_nona = final_data_nona.drop('FinalData', axis=1)
final_data_nona.head()

In [ ]:
#removing excess punctuation
all_sample_data_21_23 = final_data_nona.copy()
all_sample_data_21_23.columns = all_sample_data_21_23.columns.str.replace('[a-zA-Z]+\\.','', regex=True)
all_sample_data_21_23.head()

In [ ]:
pd.set_option('display.max_rows', 5)

#all_sample_data_21_23['SampleDate'] = pd.to_datetime(all_sample_data_21_23.SampleDate, format='%Y-%m-%d %H:%M:%S')
all_sample_data_21_23['SampleDate'] = pd.to_datetime(all_sample_data_21_23['SampleDate'], utc=True)
all_sample_data_21_23['SampleDate'] = all_sample_data_21_23['SampleDate'].dt.strftime('%Y-%m-%d')
all_sample_data_21_23 = all_sample_data_21_23.sort_values(['SampleDate'], ascending=True)

#sort values
all_sample_data_21_23 = all_sample_data_21_23.sort_values(['InstallationName'], ascending=True)

#remove meaningless columns
all_sample_data_21_23 = all_sample_data_21_23.drop('FinalResultDate', axis=1)
all_sample_data_21_23 = all_sample_data_21_23.drop('Id', axis=1)
all_sample_data_21_23 = all_sample_data_21_23.drop('LabReportId', axis=1)
all_sample_data_21_23 = all_sample_data_21_23.drop('LaboratorySampleId', axis=1)
all_sample_data_21_23['InstallationName'] = all_sample_data_21_23['InstallationName'].str.upper()
all_sample_data_21_23.reset_index(drop=True, inplace=True)
all_sample_data_21_23['LkupInstallationId'] = all_sample_data_21_23['LkupInstallationId'].astype(int)
all_sample_data_21_23.head()

In [ ]:
#list number of unique items in a column
unique = all_sample_data_21_23['PreOrPostTreatment'].unique()
unique

In [ ]:
#rename columns to standardize column names
all_sample_data_21_23 = all_sample_data_21_23.rename(columns={'Component': 'branch',
                                                              'InstallationName': 'name',
                                                              'InstallationType':'type',
                                                              'SampleDate':'date',
                                                              'AnalysisMethod':'analysis_method',
                                                              'CasNumber':'cas_number',
                                                              'AnalyteAbbrev':'analyte',
                                                              'FinalResult':'results',
                                                              'LimitOfDetection':'limit_of_detection',
                                                              'FinalUnits':'units',
                                                              'FinalQualification':'qualifier',
                                                              'LkupInstallationId':'lkup_installation_id',
                                                              'TreatmentSystem':'treatment_system',
                                                              'PreOrPostTreatment':'pre_or_post_treatment'})


all_sample_data_21_23['name'] = all_sample_data_21_23['name'].apply(lambda x: x.replace('(','').replace(')','')) 

#change empty cells to numpy nan format. Since there were no "nan" values in the dataset, 
#the blanks are assumed to be "nan" /non-detect
all_sample_data_21_23["results"]  = all_sample_data_21_23["results"].replace('', np.nan)

all_sample_data_21_23.tail()

In [ ]:
#determine number of rows, columns
all_sample_data_21_23.shape

In [ ]:
#count nubmer of unique military bases 
all_sample_data_21_23['name'].nunique()

In [ ]:
all_sample_data_21_23['pre_or_post_treatment'].unique()

In [ ]:
all_sample_data_21_23.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\all_sample_data_21_23.csv", index=False, na_rep=np.nan)                              
                            

### Clean and filter data

In [ ]:
#determine number of individual bases
pd.set_option('display.max_rows', 80)
bases = final_data_nona.groupby('FinalData.InstallationName', as_index=False).count()

bases.tail()

In [ ]:
#pull out only PFAS chemicals that EPA has released updated standards for
#https://www.epa.gov/sdwa/and-polyfluoroalkyl-substances-pfas
EPA_chemicals = ['PFOA','PFOS','PFNA','PFHxS','PFBS','HFPO-DA'] #HFPO-DA is Gen X
final_data_EPA = all_sample_data_21_23[all_sample_data_21_23['analyte'].isin(EPA_chemicals)]

final_data_EPA.reset_index(drop=True, inplace=True)
final_data_EPA.head()

In [ ]:
final_data_EPA.shape

In [ ]:
final_data_EPA['name'].nunique()

In [ ]:
final_data_EPA.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\EPA_PFAS_data_21_23.csv", index=False, na_rep=np.nan)


In [ ]:
#select only PFOA and PFOS because EPA has proposed standards for these that aren't based on hazard index
PFOS_PFOA = ['PFOA', 'PFOS']
final_PFOS_PFOA = final_data_EPA[final_data_EPA['analyte'].isin(PFOS_PFOA)]
final_PFOS_PFOA.reset_index(drop=True, inplace=True)
#final_PFOS_PFOA = final_PFOS_PFOA.sort_values(['SampleDate'], ascending=True)
final_PFOS_PFOA.head()

In [ ]:
#calculate number of unique facilities in 2021-2023 dataset
final_PFOS_PFOA["name"].nunique()

In [ ]:
#optional export to csv. Other files below contain same information
#final_PFOS_PFOA.to_csv(r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\PFOA_PFOS_21_23.csv", index=False)

### Compare samples to EPA proposed standard of 4 ppt

In [ ]:
#add column with EPA proposed MCL of 4 ppt (ng/L) (same for both PFOA and PFOS) for comparison
#From EPA: Maximum Contaminant Level (MCL): The highest level of a contaminant that is allowed in drinking water
final_PFOS_PFOA.insert(8, 'screening_level', 4)
pd.set_option('display.max_rows', 500)
final_PFOS_PFOA.head()

In [ ]:
#compare sampling results with EPA's proposed MCL of 4 ppt
final_PFOS_PFOA_4 = final_PFOS_PFOA.copy()
final_PFOS_PFOA_4["results"] = pd.to_numeric(final_PFOS_PFOA_4["results"])
final_PFOS_PFOA_4.insert(9, "exceedance", final_PFOS_PFOA_4["results"] > final_PFOS_PFOA_4["screening_level"])
pd.set_option('display.max_rows', 500)
final_PFOS_PFOA_4.head()

In [ ]:
final_PFOS_PFOA_4.shape

In [ ]:
final_PFOS_PFOA['name'].nunique()

In [ ]:
final_PFOS_PFOA_4.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\PFOA_PFOS_4MCL_21_23.csv", index=False, na_rep=np.nan)

In [ ]:
#select bases that exceed the standard
exceed = final_PFOS_PFOA_4.loc[final_PFOS_PFOA_4['exceedance'] == True]
exceed.reset_index(drop=True, inplace=True)
pd.set_option('display.max_rows', 500)
exceed.head()

In [ ]:
#determine number of bases with exceedances
bases_exceed = exceed.groupby('name', as_index=False).count()
bases_exceed.head()

In [ ]:
#determine nubmer of bases with exceedances and no treatment system
exceed_no_treat = final_PFOS_PFOA.loc[final_PFOS_PFOA['treatment_system'] == 'No']
exceed_no_treat = exceed_no_treat.groupby('name', as_index=False).count()
exceed_no_treat.head()

In [ ]:
final_PFOS_PFOA.head()

### Only keep the highest concentration for each military base for ease of visualization on a map

In [ ]:
#sorting by military base name and then PFAS concentration and then only keeping the highest concentration of PFAS for
#each military base

max_final_PFOS_PFOA = final_PFOS_PFOA_4.sort_values(['name','results'],
                                                  ascending=False).drop_duplicates('name')
max_final_PFOS_PFOA = max_final_PFOS_PFOA.sort_values('name', ascending=True)

max_final_PFOS_PFOA.reset_index(drop=True, inplace=True)
max_final_PFOS_PFOA.head()

In [ ]:
#calculate summary statistics for the 2021-2023 dataset
max_final_PFOS_PFOA[["results"]].describe()

In [ ]:
#calculate median of PFAS concentrations
max_final_PFOS_PFOA[["results"]].median()

In [ ]:
#create histogram of frequency distribution of concentrations
fig, ax = plt.subplots(figsize=(20, 10))
plt.axvline(x=4, color = 'red') 
plt.axvline(x=70, color = 'red') 
plot_max_final_PFOS_PFOA = plt.hist(max_final_PFOS_PFOA['results'], bins=50, linewidth=0.5, edgecolor="white")
plt.suptitle('Frequency distribution of highest concentration of PFAS reported on military installations, 2021-2023',
             fontsize=20)
plt.xlabel('results (ppt)', fontsize=20)
plt.ylabel('frequency', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=20, width=2, length=4)
plt.locator_params(axis='y', nbins=5)

plot_max_final_PFOS_PFOA

### Add location data to the sample data 

In [ ]:
#make name of facilities in spatial dataframe all uppercase
metadata_df['InstallationName'] = metadata_df['InstallationName'].str.upper()
#drop excess columns
metadata_df = metadata_df.drop(['Id','DodComponentId'], axis=1)
metadata_df.reset_index(drop=True, inplace=True)

#rename column names
metadata_df = metadata_df.rename(columns={'DodComponent': 'branch',
                                          'State':'state',
                                          'InstallationName':'name',
                                          'InstallationType':'type',
                                          'Latitude': 'latitude',
                                          'Longitude': 'longitude'})

#remove paranthesis from names
metadata_df['name'] = metadata_df['name'].apply(lambda x: x.replace('(','').replace(')','')) 

metadata_df.head()

In [ ]:
metadata_df.shape

In [ ]:
#determine number of unique facilities in the geospatial dataset
metadata_df["name"].nunique()

In [ ]:
metadata_df["longitude"].nunique()

In [ ]:
metadata_df.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\military_bases_spatial.csv", index=False, na_rep=np.nan)

In [ ]:
#merge sample data and metadata df to attach latitude and longitude data from the metadata to the sample data
max_final_PFOS_PFOA_geo = max_final_PFOS_PFOA.merge(
    metadata_df[['name', 'state','latitude','longitude']], on='name', how='left', indicator=True)
#remove extra names from column names
max_final_PFOS_PFOA_geo.columns = max_final_PFOS_PFOA_geo.columns.str.replace('[a-zA-Z]+\\.','', regex=True)
#drop columns that are not needed
max_final_PFOS_PFOA_geo = max_final_PFOS_PFOA_geo.drop(['_merge',
                                                       # 'Id',
                                                        'type'], axis=1)

#remove time stamps and format dates
#max_final_PFOS_PFOA_geo['date'] = pd.to_datetime(max_final_PFOS_PFOA_geo.date, format='%Y-%m-%d %H:%M:%S')
max_final_PFOS_PFOA_geo['date'] = pd.to_datetime(max_final_PFOS_PFOA_geo['date'], utc=True)
max_final_PFOS_PFOA_geo['date'] = max_final_PFOS_PFOA_geo['date'].dt.strftime('%Y-%m-%d')
# make military base name all uppercase to make name formats consistent
max_final_PFOS_PFOA_geo['name'] = max_final_PFOS_PFOA_geo['name'].str.upper()
#change location of the state column
max_final_PFOS_PFOA_geo.insert(0, 'state', max_final_PFOS_PFOA_geo.pop('state'))

#remove trailing zeros from result column
max_final_PFOS_PFOA_geo["results"] = max_final_PFOS_PFOA_geo["results"].apply(
    lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)

#add state for Manchester Fuel Depot
pd.options.mode.chained_assignment = None
max_final_PFOS_PFOA_geo['state'].loc[max_final_PFOS_PFOA_geo['name'] == 'MANCHESTER WA FUELDPTPSND'] = 'Washington'

max_final_PFOS_PFOA_geo.head()

In [ ]:
#list the unique military installation names
max_final_PFOS_PFOA_geo['name'].unique

In [ ]:
#determine number of exceedances above 4 ppt
max_final_PFOS_PFOA_geo.exceedance.value_counts()

In [ ]:
max_final_PFOS_PFOA_geo.shape

In [ ]:
max_final_PFOS_PFOA_geo['name'].nunique()

In [ ]:
max_final_PFOS_PFOA_geo.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\geo_max_PFOA_PFOS_4MCL_21_23.csv", index=False, na_rep=np.nan)

In [ ]:
#filter data to only display exceedances to prepare for mapping
exceed_only_spatial = max_final_PFOS_PFOA_geo.loc[max_final_PFOS_PFOA_geo['exceedance'] == True]
exceed_only_spatial.head() #.dtypes

In [ ]:
#convert results column to numeric to allow for mapping
exceed_only_spatial_map = exceed_only_spatial.copy()
exceed_only_spatial_map['results'] = pd.to_numeric(exceed_only_spatial["results"])

In [ ]:
#create map of the data
fig = px.scatter_geo(exceed_only_spatial_map,
    lon='longitude',
    lat='latitude',
    size='results',
    color_discrete_sequence=["red"],
    hover_name="name",
    title = 'Military installations with PFAS exceedances in drinking water of EPA proposed standard of 4 ppt, 2021-2023',
)


fig.update_layout(
    geo_scope='usa',
  #  margin=dict(l=50, r=50, t=20, b=20)
    
)

fig

### Compare sample results to 70 ppt rather than 4 ppt
#### EPA has older guidance of 70 ppt

In [ ]:
#applying the 70 ppt standard to PFOS and PFOA data instead of 4 ppt
final_PFOS_PFOA_70 = final_PFOS_PFOA.copy()
final_PFOS_PFOA_70['screening_level'] = 70
final_PFOS_PFOA_70["results"]  = final_PFOS_PFOA_70["results"] .replace('nan', np.nan)
final_PFOS_PFOA_70["results"] = pd.to_numeric(final_PFOS_PFOA_70["results"])
final_PFOS_PFOA_70["exceedance"] = np.where(final_PFOS_PFOA_70['results']>= final_PFOS_PFOA_70['screening_level'], 
                                            True, False)
#remove trailing zeros
final_PFOS_PFOA_70["results"] = final_PFOS_PFOA_70["results"].apply(
   lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)

final_PFOS_PFOA_70.reset_index(drop=True, inplace=True)

final_PFOS_PFOA_70.head()

In [ ]:
final_PFOS_PFOA_70.shape

In [ ]:
final_PFOS_PFOA_70['name'].nunique()

In [ ]:
final_PFOS_PFOA_70.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\PFOA_PFOS_70MCL_21_23.csv", index=False, na_rep=np.nan)

In [ ]:
max_final_PFOS_PFOA_70_geo = max_final_PFOS_PFOA_geo.copy(deep=True)
max_final_PFOS_PFOA_70_geo['screening_level'] = 70
max_final_PFOS_PFOA_70_geo["results"]  = max_final_PFOS_PFOA_70_geo["results"] .replace('nan', np.nan)
max_final_PFOS_PFOA_70_geo["results"] = pd.to_numeric(max_final_PFOS_PFOA_70_geo["results"])
max_final_PFOS_PFOA_70_geo["exceedance"] = np.where(max_final_PFOS_PFOA_70_geo['results']>= 
                                                    max_final_PFOS_PFOA_70_geo['screening_level'], True, False)
#remove trailing zeros
final_PFOS_PFOA_70["results"] = final_PFOS_PFOA_70["results"].apply(
   lambda x:'{:.{}f}'.format(x, (len(str(x)) - 1 - int(str(x).find('.')))) if isinstance(x, float) else x)

max_final_PFOS_PFOA_70_geo.reset_index(drop=True, inplace=True)

pd.set_option('display.max_rows', 500)
max_final_PFOS_PFOA_70_geo.head()

In [ ]:
#calculate number of facilities with an exceedance of EPA MCL old screening level of 70 ppt
max_final_PFOS_PFOA_70_geo.exceedance.value_counts()

In [ ]:
max_final_PFOS_PFOA_70_geo.shape

In [ ]:
max_final_PFOS_PFOA_70_geo['name'].nunique()

In [ ]:
max_final_PFOS_PFOA_70_geo['longitude'].nunique()

In [ ]:
max_final_PFOS_PFOA_70_geo.to_csv(
r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\Jan_2024\geo_max_PFOA_PFOS_70MCL_21_23.csv", index=False, na_rep=np.nan)

#### Look at all the data to determine date range

In [ ]:
#analyze dates
final_data_dates = all_sample_data_21_23
final_data_dates = final_data_dates.sort_values(['date'], ascending=True)
final_data_dates.reset_index(drop=True, inplace=True)
final_data_dates.head()

In [ ]:
final_data_dates.tail()